In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import shutil

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False
# 매직명령어 => 쥬피터노트북에서 그래프 삽입 기능 
%matplotlib inline
# 글꼴 선명화 
%config InlineBackend.figure_format = 'retina'

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
# 신경망 모델을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형 회귀 레이어
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 객체
from tensorflow.keras.layers import Activation

# CNN : 커널을 통해 합성곱을 구하는 것. 이미지의 특징이 두드러 지게 한다.
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv1D

# MaxPooling : 커널내에서 가장 큰 값을 추출하는 방식으로 불필요한 부분을 제거한다.
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPool1D

# 1차원으로 변환하는 것
from tensorflow.keras.layers import Flatten

# Dropout : 이미자나 영상, 음파 등의 데이터는 오랫동안 학습을 진행 시켜야 한다.
# 하지만 너무 빨리 과적합이 발생되면 조기 중단 때문에 학습 횟수가 줄어들 수 있다.
# 이에 은닉의 노드를 일부 사용하지 않으므로써 과적합이 빨리 오는 것을 예방하고
# 오랫동안 학습이 가능하다.
from tensorflow.keras.layers import Dropout

# Embadding : 단어의 수를 조정한다.
from tensorflow.keras.layers import Embedding

# RNN(LSTM)
from tensorflow.keras.layers import LSTM

# GAN
# 데이터 표준화
from tensorflow.keras.layers import BatchNormalization
# 데이터를 다시 구성하기 위한 것
from tensorflow.keras.layers import Reshape
# 업샘플링
from tensorflow.keras.layers import UpSampling2D
# 이미지 생성을 위해 사용할 데이터 공간을 생성
from tensorflow.keras.layers import Input
# 다수의 신경망을 연결하기 위한 것(생성자와 판별자를 연결)
from tensorflow.keras.models import Model

# 원핫 인코딩을 수행하는 함수
from tensorflow.keras.utils import to_categorical

# 저장된 학습모델을 복원한다.
from tensorflow.keras.models import load_model

# 모델을 자동 저장한다.
from tensorflow.keras.callbacks import ModelCheckpoint
# 성능이 더이상 좋아지지 않을 경우 중단 시킨다.
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# 문장을 잘라준다.
from tensorflow.keras.preprocessing.text import Tokenizer
# 모든 문장 데이터의 단어 데이터의 수를 동일한 수로 맞춰준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문자열을 가지고 단어 사전을 만들어준다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 문자열 => 숫자
from sklearn.preprocessing import LabelEncoder

# 전체데이터를 학습용과 검증으로 나눈다.
from sklearn.model_selection import train_test_split

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

### GAN
- 생성자가 만든 이미지를 판별자가 평가하여 원본 이미지와 비교하여 얼마나 유사한지를 평가하고 평가한 결과를 다시 생성자에게 전달하여 생성과 평가를 많이 진행하면 원본 이미지와 유사한 이미지를 생성하는 딥러닝 기법
- 세상에 존재하지 않는 이미지를 만든다.
- 생성자 역할을 할 신경망과 판별자 역할을 할 신경망이 있어야 한다.

In [2]:
# MNIST
from tensorflow.keras.datasets import mnist

In [3]:
(X_train, _), (_, _) = mnist.load_data()

In [4]:
# 이미지 개수, 세로, 가로 형태의 행렬을
# 이미지 개수, 세로, 가로, 채널 형태의 행렬로 변환한다.
# 채널 : 흑백1, 컬러3
cnt = X_train.shape[0]
h = X_train.shape[1]
w = X_train.shape[2]
channel = 1
X_train = X_train.reshape(cnt, h, w, channel).astype('float64')
X_train.shape

(60000, 28, 28, 1)

In [5]:
# 이미지 노멀라이징
# 생성자의 마지막 활성화 하수는 tanh을 사용하는데 tanh은 -1 ~ 1 사이로 맞춘다.
# 이에 원본 이미지 데이터도 -1 ~ 1 사이로 설정한다.
X_train = (X_train -127.5) / 127.5
# X_train

In [6]:
# 생성자 신경망 설정
generator = Sequential()

# 첫 번째 은닉층
# 7 * 7 : 최초로 생성되는 이미지의 크기
# 7 * 7 -> 14 * 14 -> 28 * 28
# input_dim : 나중에 지정할 임의 벡터의 개수. 임의의 벡터는
# 그림을 그릴 도화지에 해당한다.
generator.add(Dense(128 * 7 * 7, input_dim=100))
generator.add(Activation('relu'))

# 새롭게 랜덤 생성된 이미지 데이터를 표준화한다.
generator.add(BatchNormalization())

# CNN을 통해 작업을 할때는 데이터를 세로, 가로, 임의의 노드수 형태로 변경해준다.
generator.add(Reshape((7, 7, 128)))
# 두배로 증가
generator.add(UpSampling2D())

# CNN 학습
# padding을 same으로 설정하여 CNN 학습시 데이터가 작아지는 것을 방지한다.
# 노드의 수는 위에서 설정한 임의의 노드의 절반으로 줄여주는 것을 권한다.
generator.add(Conv2D(64, kernel_size=3, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation('relu'))

# 두배로 증가
generator.add(UpSampling2D())

# CNN 학습
generator.add(Conv2D(1, kernel_size=3, padding='same'))
generator.add(Activation('tanh'))

In [7]:
# 판별자
discriminator = Sequential()

# CNN 학습
# 생성자의 마지막 은닉층 직전의 은닉층 노드가 64이므로...
# 생성자에서 최종 생성되 이미지가 28*28 이므로...
discriminator.add(Conv2D(64, kernel_size=3, input_shape=(28,28,1), padding='same'))
discriminator.add(Activation('relu'))
discriminator.add(Dropout(0.3))

discriminator.add(Conv2D(128, kernel_size=3, padding='same'))
discriminator.add(Activation('relu'))
discriminator.add(Dropout(0.3))

# 예측결과 추출을 위해 차원으로 변환한다.
discriminator.add(Flatten())

# 출력층
# 출력층의 결과는 유사도 값 하나가 전달된다.
discriminator.add(Dense(1))
discriminator.add(Activation('sigmoid'))

In [8]:
# 판별자 컴파일
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
# 판별자는 학습하지 않도록 설정
# 판별자가 학습을 하게 되면 진짜 이미지를 새롭게 생성된 이미지와
# 유사하게 만들려고 노력한다. 이에 판별자는 학습을 하면 안된다.
discriminator.trainable = False

In [9]:
# 생성자와 판별자를 연결하는 gan 모델을 생성한다.
# 생성자
# 생성자에서 사용할 임의의 백터 100개를 
ginput = Input(shape=(100,))
# 판별자에게 생성자를 지정한다.
dis_output = discriminator(generator(ginput))

In [10]:
# GAN 모델 생성
gan = Model(ginput, dis_output)

In [11]:
# 최종 모델 컴파일
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         733185    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 174849    
                                                                 
Total params: 908,034
Trainable params: 720,513
Non-trainable params: 187,521
_________________________________________________________________


In [12]:
# 한번에 생성할 이미지의 개수
batch_size = 32
# 잘 만들어진 이미지 정보를 가질 행렬
true = np.ones((batch_size, 1))
# 가짜 이미지 정보를 가질 행렬
fake = np.zeros((batch_size, 1))

display(true)
display(fake)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [ ]:
epochs = 20001

for i in range(epochs):
    # 실제 데이터를 파별자에 입력하는 부분
    # 전체 데이터에서 램덤하게 32개를 추출하여 담아준다.
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    
    # 추출한 이미지와 true 행렬에 들어있는 값을 이용하여 손실률을 파악한다.
    d_loss_real = discriminator.train_on_batch(imgs, true)
    
    # 가상 이미지를 판별자에 입력하는 부분
    # 0 ~ 1사이의 값을 램덤하게 추출하여 100개짜리 크기의 batch_size 수 만큼 생성한다.
    noise = np.random.normal(0, 1, (batch_size, 100))
    gen_imgs = generator.predict(noise)
    
    # 새롭게 생성된 이미지와 fake 행렬에 들어잇는 값을 이용하여 손실률을 파악한다.
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    
    # 판별자와 생성자의 오차를 계산한다.
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    g_loss = gan.train_on_batch(noise, true)
    
    print(f'epochs : {i}, d_loss : {d_loss}, g_loss : {g_loss}')
    
    # 200번 마다 한번씩 저장한다.
    if i % 200 == 0 :
        # 랜덤 이미지를 생성한다.
        noise = np.random.normal(0, 1, (25, 100))
        # 예측 이미지를 생성한다.
        gen_img = generator.predict(noise)
        
        # 그리기 위해 생성된 이미지 값을 0 ~ 1 사이로 조정한다.
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        # 5* 5 짜리 그래프 영역을 생성한다.
        fig, axs = plt.subplots(5, 5)
        
        # 생성된 이미지를 그린다.
        image_idx = 0
        for j in range(5) : 
            for k in range(5) :
                # 이미지를 그린다.
                axs[j, k].imshow(gen_imgs[image_idx, :, :, 0], cmap='gray')
                axs[j, k].axis('off')
                image_idx = image_idx + 1
                
        # 파일로 저장한다.
        fig.savefig(f'./gan_image/gan_nmist_{i}.png')

1/1 [==============================] - 0s 134ms/step
epochs : 1, d_loss : 0.8638397753238678, g_loss : 0.04554937034845352
1/1 [==============================] - 0s 12ms/step
epochs : 1, d_loss : 0.4866454194416292, g_loss : 0.018085286021232605
1/1 [==============================] - 0s 13ms/step
epochs : 1, d_loss : 0.4158308797632344, g_loss : 0.008793025277554989
1/1 [==============================] - 0s 12ms/step
epochs : 1, d_loss : 0.39856062641047174, g_loss : 0.007489314768463373
1/1 [==============================] - 0s 11ms/step
epochs : 1, d_loss : 0.42272517073433846, g_loss : 0.016361162066459656
1/1 [==============================] - 0s 11ms/step
epochs : 1, d_loss : 0.4284036570461467, g_loss : 0.07203763723373413
1/1 [==============================] - 0s 12ms/step
epochs : 1, d_loss : 0.3848811322823167, g_loss : 0.36103498935699463
1/1 [==============================] - 0s 12ms/step
epochs : 1, d_loss : 0.44663814455270767, g_loss : 0.23880954086780548
1/1 [===========